code in part inspired by: https://github.com/EthanRosenthal/torchmf

In [1]:
import os
import mxnet as mx
from mxnet import gluon, nd, ndarray

import pandas as pd
import numpy as np

In [2]:
data_path = '/home/ubuntu/mxnet-the-straight-dope/incubator-mxnet/example/recommenders/ml-100k/'
num_emb = 64
opt = 'Adam'
lr = 0.02
mmntm = 0.
wd = 0.
batch_size = 50
ctx = mx.gpu()

In [3]:
def download_ml_data(prefix):
    if not os.path.exists("%s.zip" % prefix):
        print("Downloading MovieLens data: %s" % prefix)
        os.system("wget http://files.grouplens.org/datasets/movielens/%s.zip" % prefix)
        os.system("unzip %s.zip" % prefix)    

In [4]:
def max_id(fname):
    mu = 0
    mi = 0
    with open(fname) as f:
        for line in f:
            tks = line.strip().split('\t')
            if len(tks) != 4:
                continue
            mu = max(mu, int(tks[0]))
            mi = max(mi, int(tks[1]))
    return mu + 1, mi + 1
max_users, max_items = max_id(data_path + 'u.data')

In [5]:
train_df = pd.read_csv(data_path+'u1.base', header=None, sep='\t')
test_df = pd.read_csv(data_path+'u1.test', header=None, sep='\t')

train_data = nd.array(train_df[[0,1]].values, dtype=np.float32)
train_label = nd.array(train_df[2].values, dtype=np.float32)

test_data = nd.array(test_df[[0,1]].values, dtype=np.float32)
test_label = nd.array(test_df[2].values, dtype=np.float32)

In [6]:
class SparseMatrixDataset(gluon.data.Dataset):
    def __init__(self, data, label):
        assert data.shape[0] == len(label)
        self.data = data
        self.label = label
        if isinstance(label, ndarray.NDArray) and len(label.shape) == 1:
            self._label = label.asnumpy()
        else:
            self._label = label       
        
    def __getitem__(self, idx):
        return self.data[idx, 0], self.data[idx, 1], self.label[idx]
    
    def __len__(self):
        return self.data.shape[0]
        

In [7]:
class MFBlock(gluon.Block):
    def __init__(self, max_users, max_items, num_emb, dropout_p=0.5):
        super(MFBlock, self).__init__()
        
        self.max_users = max_users
        self.max_items = max_items
        self.dropout_p = dropout_p
        self.num_emb = num_emb
        
        with self.name_scope():
            self.user_embeddings = gluon.nn.Embedding(max_users, num_emb)
            self.item_embeddings = gluon.nn.Embedding(max_items, num_emb)
            self.dropout = gluon.nn.Dropout(dropout_p)
            self.dense = gluon.nn.Dense(num_emb, activation='relu')
            
    def forward(self, users, items):
        a = self.user_embeddings(users)
        a = self.dense(a)
        
        b = self.item_embeddings(items)
        b = self.dense(b)

        predictions = self.dropout(a) * self.dropout(b)      
        predictions = nd.sum(predictions, axis=1)
        return predictions

        

In [8]:
net = MFBlock(max_users=max_users, max_items=max_items, num_emb=num_emb, dropout_p=0.)
net.collect_params()

mfblock0_ (
  Parameter mfblock0_embedding0_weight (shape=(944, 64), dtype=<class 'numpy.float32'>)
  Parameter mfblock0_embedding1_weight (shape=(1683, 64), dtype=<class 'numpy.float32'>)
  Parameter mfblock0_dense0_weight (shape=(64, 0), dtype=<class 'numpy.float32'>)
  Parameter mfblock0_dense0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
)

In [9]:
loss_function = gluon.loss.L2Loss()

In [10]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx, force_reinit=True)

In [11]:
trainer = gluon.Trainer(net.collect_params(), 'sgd',
                        {'learning_rate': lr, 'wd': wd, 'momentum': 0.9})

In [12]:
train_data_iter = gluon.data.DataLoader(SparseMatrixDataset(train_data, train_label), 
                                        shuffle=True, batch_size=batch_size)
test_data_iter = gluon.data.DataLoader(SparseMatrixDataset(test_data, test_label),
                                          shuffle=True, batch_size=batch_size)

In [15]:
def eval_net(data, net):
    acc = mx.metric.RMSE()
    for i, (user, item, label) in enumerate(data):
        user = user.as_in_context(ctx).reshape((batch_size,))
        item = item.as_in_context(ctx).reshape((batch_size,))
        label = label.as_in_context(ctx).reshape((batch_size,))
        predictions = net(user, item)
        loss = loss_function(predictions, label)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [16]:
eval_net(test_data_iter, net)


3.5273488330036402

In [17]:
epochs = 10
#smoothing_constant = 10

def train(data_iter, net):
    for e in range(epochs):
        print("epoch: {}".format(e))
        for i, (user, item, label) in enumerate(train_data_iter):
            user = user.as_in_context(ctx).reshape((batch_size,))
            item = item.as_in_context(ctx).reshape((batch_size,))
            label = label.as_in_context(ctx).reshape((batch_size,))
            with mx.autograd.record():
                output = net(user, item)               
                loss = loss_function(output, label)
                loss.backward()
            net.collect_params().values()
            trainer.step(batch_size)
        print("EPOCH {}: RMSE ON TRAINING and TEST: {}. {}".format(e,
                                                                   eval_net(train_data_iter, net),
                                                                   eval_net(test_data_iter, net)))
    return "end of training"

In [18]:
train(train_data_iter, net)

epoch: 0
EPOCH 0: RMSE ON TRAINING and TEST: 0.7397429539039732. 0.7727593539834022
epoch: 1
EPOCH 1: RMSE ON TRAINING and TEST: 0.7340302160739899. 0.7680136477231979
epoch: 2
EPOCH 2: RMSE ON TRAINING and TEST: 0.7290934163123369. 0.7636049427747726
epoch: 3
EPOCH 3: RMSE ON TRAINING and TEST: 0.7489639871120453. 0.7812525823950768
epoch: 4
EPOCH 4: RMSE ON TRAINING and TEST: 0.7219638488218189. 0.7587113852620124
epoch: 5
EPOCH 5: RMSE ON TRAINING and TEST: 0.7221305305734277. 0.7613962271451951
epoch: 6
EPOCH 6: RMSE ON TRAINING and TEST: 0.6964019835107028. 0.7500802919447422
epoch: 7
EPOCH 7: RMSE ON TRAINING and TEST: 0.7040959010727703. 0.7654881411790848
epoch: 8
EPOCH 8: RMSE ON TRAINING and TEST: 0.662088219345361. 0.7430168891996145
epoch: 9
EPOCH 9: RMSE ON TRAINING and TEST: 0.641668664816767. 0.7430033217519522


'end of training'

In [38]:
net1 = gluon.nn.Sequential()
with net1.name_scope():
    net1.add(gluon.nn.Embedding(max_users, num_emb))
    net1.add(gluon.nn.Dense(64))
    
net1.collect_params()

sequential6_ (
  Parameter sequential6_embedding0_weight (shape=(944, 64), dtype=<class 'numpy.float32'>)
  Parameter sequential6_dense0_weight (shape=(64, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential6_dense0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
)

In [39]:
train_df = pd.read_csv(data_path+'u1.base', header=None, sep='\t')
test_df = pd.read_csv(data_path+'u1.test', header=None, sep='\t')

user_train_data = nd.array(train_df[[0]].values, dtype=np.float32)
item_train_data = nd.array(train_df[[1]].values, dtype=np.float32)
train_label = nd.array(train_df[2].values, dtype=np.float32)

user_test_data = nd.array(test_df[[0]].values, dtype=np.float32)
item_test_data = nd.array(test_df[[1]].values, dtype=np.float32)
test_label = nd.array(test_df[2].values, dtype=np.float32)



In [ ]:
user_train_data_iter = gluon.data.DataLoader(gluon.data.ArrayDataset())